# Data Wrangling with OpenStreetMaps and SQL
<a href="http://www.openstreetmap.org">OpenStreetMap</a> is a volunteered geographic information project that runs on open source collaboration to provide free and editable maps to everyone on this planet (assuming they have internet access). 
<p>As expected with user generated content, inconsistencies and errors (misspellings, innacurate info, etc) in the data are present. The goal of this project is to:
<ul>
<li>Download map data of a selected city</li>
<li>Import the data into an sqlite database</li>
<li>Correct the problems and fix errors</li>
</ul>

## Map Area

<b>The island of O'ahu, HI, United States</b> (<a href="http://www.openstreetmap.org/relation/3489649">Open Street Map Relation: Honolulu, HI</a>)
<p>I chose O'ahu for several reasons, among which are my intense affinity for the area from having spend considerable time there.</p>

## Process

This project will include three steps: 
<ol>
<li><b>Data Review:</b> In which the OpenStreetMap .xml file is audited and the data types cleaned in preparation for analysis</li>
<li><b>SQL Preparation:</b> Where the cleaned .xml file is parsed into .csv files suitable for SQL query</li>
<li><b>Query</b>: In which SQL queries are applied to answer general questions about the area, such as number of retaurants</li>
</ol>

### Contents
I have included all of the individual code files used in this project as part of the submission package:  
<ol>
<li><b>oahu_audit.py</b> Gather list of users by uid</li>
<li><b>oahu_street_clean.py</b> Standardize street names</li>
<li><b>oahu_zip_clean.py</b> Standardize zip codes</li>
<li><b>oahu_phone_clean.py</b> Standardize phone numbers</li>
<li><b>oahu_tag_clean.py</b> Cleans any troublesome tags in the data set</li>
<li><b>oahu_csv_convert.py</b> Convert the .xml file into a series of .csv files</li>
<li><b>oahu_schema.py</b> Create the proper shcemas</li>
<li><b>oahu_sql_prep.py</b> Prepares the files for SQL queries</li>
<li><b>oahu_sql_queries</b> A series of SQL queries run on the converted .csv files</li>
</ol>

All of the actual data wrangling, cleaning, queries, and results were obtained by running the included <b>P3_O'ahu_Code.ipynb</b> file.

## Overview of the Data

Before we begin, let's get a summary of the file sizes for all of the data used in this project<br/><br/>
File Sizes:

In [1]:
import os
print 'The file downloaded for the map data of the city of Honolulu, HI is {} MB'.format(os.path.getsize("c:\honolulu_hawaii.osm")/1.0e6)
print 'The csv file for nodes is {} MB'.format(os.path.getsize("nodes.csv")/1.0e6)
print 'The csv file for nodes_tags is {} MB'.format(os.path.getsize("nodes_tags.csv")/1.0e6)
print 'The csv file for ways is {} MB'.format(os.path.getsize("ways.csv")/1.0e6)
print 'The csv file for ways_nodes is {} MB'.format(os.path.getsize("ways_nodes.csv")/1.0e6)
print 'The csv file for ways_tags is {} MB'.format(os.path.getsize("ways_tags.csv")/1.0e6)
print 'The db file for Oahu OpenStreetMap  is {} MB'.format(os.path.getsize("OpenStreetMap_Oahu.db")/1.0e6)

The file downloaded for the map data of the city of Honolulu, HI is 61.975581 MB
The csv file for nodes is 24.392537 MB
The csv file for nodes_tags is 0.631892 MB
The csv file for ways is 1.947941 MB
The csv file for ways_nodes is 8.52166 MB
The csv file for ways_tags is 3.866938 MB
The db file for Oahu OpenStreetMap  is 35.2 MB


## Data Review
My initial data review involved checking
<ul>
<li>Street Names (inlcuding abbreviations)</li>
<li>Postal Codes</li>
<li>Phone Numbers</li>
</ul>

The goal was to assess the consistency and quality for these three elements, and to standardize the information. 

### Problem with the Data - Standardization
Among the expected problems with the data were inconsistent street name abbreviations, such as Bl, Bl. Blvd, and Blvd. for "Boulevard".  I also expect similar inconsistencies in the formatting and presentation of zip codes and phone numbers.

To mitigate this, I will standardize all three by first identifying the errors, then correcting them by:
<ol>
<li>Create a list of "standardized" street names/zip code/phone number</li>
<li>Audit the data to identify non-standardized abbreviations, and to replace them with the standardized forms</li>
</ol>

Using street names as an example, the process will involve creating a function "audit_street_type" which collects the last word in the "street_name" string.  Any words not present in the original list of standardized names will be added to the "street_types" dictionary, and will eventually be corrected by the "update_name_function.

The code to perform these tasks are contained in the "oahu_street_clean.py", "oahu_clean_zip.py", and "oahu_clean_phone.py" files

<b>Street Standardization</b>
Code sample:

![alt text](street_correction.png "Street Correction")

This updated all substrings, such that: “Ena Rd” becomes “Ena Road”. It is important to note that this does not mean, all street adresses on the island of O'ahu have been cleaned. Our expected list in the regex and anticipiated alternate street type endings are very likely incomplete.  However, this is a good start.

<b>Zip Code Standardization</b>
Code sample:

![alt text](zip_correction.png "Zip Code Correctione")

As we see, all the zip codes now conform to our five-digit standard, and any +4 extensions or instances in which "HI" was present have been removed.

<b>Phone Number Standardization</b>
Code sample:

![alt text](phone_correction.png "Phone Correctione")

Once again the results are that any oddly formatted phone numbers, such as having "1" at the front, or the area code in parentheses such as (808), or using dashes or dots to separate the digits (i.e. 808.555.1212) are removed.

## SQL Queries

We're now ready to use the created .csv files for sql queries: nodes_path, node_tags, ways, ways_nodes and ways_tags

In [2]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = "OpenStreetMap_Oahu.db"
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes')
conn.commit()

cur.execute('''
    Create Table nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')

conn.commit()

### Number of Nodes

![alt text](num_nodes.png "NUmber of Nodes")

### Number of Ways

![alt text](num_ways.png "Number of Ways")

### Number of Unique Users

![alt text](num_unique.png "Number of Unique Users")

### Top 10 Contributors

![alt text](top_users.png "Top 10 Users")

### Number of users with fewer than 10 updates

![alt text](num_less_10.png "Number of Users Less 10 Updates")

### Number of Users with only one update
![alt text](num_once.png "Number of Users Once")

### Metropolitan areas of O'ahu

![alt text](metro_areas.png "Metro Areas")

### Top 10 Tourist ameneties

![alt text](top_tourist.png "Top 10 Tourist")

### Number of restaurants by metropolitan area

![alt text](num_restaurant.png "Restaurants by Area")

### Top 10 types of food

![alt text](top_food.png "Top 10 Food Types")

# Summary

Although this process went a long way to cleaning and standardizing the data, there are clearly several more steps that are required to ensure the data is completely accurate and internally consistent.

Further, there seems to be several instances in which data is either missing or incorrectly added.  For example, the data review showed that there are 48 restaurants in the Honolulu area, however, even a cursory web search would indicate at many times that number.  Other examples of this sort of inaccuracy are also present.

This exercise also exposed the limits of the sort of labels used in OSM datasets.  Tourist ameneties did not include Beaches, Parks, surfing supplies, snorkeling equipment rentals, or other attractions closely associated with Hawai'i.

Lastly, the OSM project is a wonderful example of both the benefits and shortcomings of depending on independent users contributing information.  This sort of "crowdsourcing" strategy provides the opportunity for anyone to contribute information that may either be missing or needs to be updated.  Unfortunately, the lack of standards or uniform requirements does result in data that is often quite messy and inconsistent.